##### Copyright 2024 Google LLC. Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

# Data Preparation

#### Step 1: Get the Embeddings: The following cells in this notebook will access the PD-Foundations embeddings directly from a BigQuery table.

⚠️ Important: To run these cells successfully, you must first obtain access to the embeddings dataset via a BigQuery Listing. Please use [this form](https://forms.gle/ysdp5uUoPrMrhjZQA) to apply for research access or to join the Early Access Program waitlist.

Once your access is approved, ensure you are authenticated in this Colab with a Google account that has permission to query the BigQuery table....

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PROJECT_ID = '<project_id>' # @param {type:"string"}

In [ ]:
import pandas as pd
import pandas_gbq


# @markdown Specify the BigQuery table ID for the county embeddings.
# Example: 'your-gcp-project.your_dataset.your_table'
bigquery_table_id = '<project_id>.pdfm_embeddings.us_embeddings_v0' # @param {type:"string"}

query = f"SELECT * FROM `{bigquery_table_id}` WHERE region_type='county'"

# Load data from BigQuery into a pandas DataFrame.
county_embeddings = pandas_gbq.read_gbq(query, project_id=PROJECT_ID, dialect='standard').set_index('place_name')

In [ ]:
# @markdown Specify the BigQuery table ID for the zcta embeddings.
bigquery_table_id = '<project_id>.pdfm_embeddings.us_embeddings_v0' # @param {type:"string"}
query = f"SELECT * FROM `{bigquery_table_id}` WHERE region_type='postal_code'"
zip_embeddings = pandas_gbq.read_gbq(query, project_id=PROJECT_ID, dialect='standard').set_index('place_name')


In [ ]:
embeddings = pd.concat([county_embeddings, zip_embeddings])

The colab uses geojson files which are available in the github repo under data folder. Download the geojson file into a local folder or a folder under Google drive. Here we assume that you have downloaded the file in Google Drive folder called pdfm_embeddings/v0/us.

In [ ]:
#@markdown Specify the path to the geo folder.
BASE_PATH = '/content/drive/MyDrive/pdfm_embeddings/v0/us/' #@param {type:"string"}



In [ ]:
embedding_features = [f'feature{x}' for x in range(330)]
embeddings.head(2)

In [ ]:
embeddings.index

#### Step 2: Download and load a few variables from data commons.

In [ ]:
!pip install datacommons_pandas --upgrade --quiet
import datacommons_pandas as dc

In [ ]:
# This may take a few minutes to run.
labels = ['Count_Person',
          'Count_Person_EducationalAttainmentBachelorsDegreeOrHigher',
          'Median_Age_Person',
          'Median_Income_Household',
          'Percent_Person_WithAsthma',
          'Percent_Person_WithHighBloodPressure'
          ]
df_labels = dc.build_multivariate_dataframe(embeddings.index, labels)
print(df_labels.shape)
df_labels.head(2)

In [ ]:
df = embeddings.join(df_labels)
df.head(1)

In [ ]:
df['state'] = df['location_metadata'].apply(lambda x: x.get('administrative_area_level1'))
df['county'] = df['location_metadata'].apply(lambda x: x.get('administrative_area_level2'))


In [ ]:
df['Percent_Person_WithHigherEdu'] = (df.Count_Person_EducationalAttainmentBachelorsDegreeOrHigher / df.Count_Person) * 100
df['county_id'] = df['county'] + df['state']

# Data Visualizations

## Download the county and zcta (Zipcode census tabulation area) level geojson file.

The county and zcta level geojson file are available in the same folder as the embeddings. Download the geojson file into a local folder or a folder under Google drive. Here we assume that you have downloaded the file in Google Drive folder called pdfm_embeddings/v0/us

In [ ]:
import geopandas as gpd
county_geo = gpd.read_file(BASE_PATH + 'county.geojson')
zip_geo = gpd.read_file(BASE_PATH + 'zcta.geojson')

In [ ]:
geo = pd.concat([county_geo, zip_geo]).rename(columns={'place': 'place_name'}).set_index('place_name')
embeddings = gpd.GeoDataFrame(embeddings, geometry=geo.geometry)
embeddings.shape

In [ ]:
embeddings.head(10)

### Map out an embedding dimension spatially

In [ ]:
def get_locale(df, index, states=None, counties=None):
  df = df[df.index.isin(index)]
  if not states and not counties:
    return df
  filter = df['location_metadata'].apply(lambda x: x.get('administrative_area_level1')).isin(states)
  if counties:
    filter &= df['location_metadata'].apply(lambda x: x.get('administrative_area_level2')).isin(counties)
  return df[filter]

In [ ]:
#@title Map out an embedding dimension feature0 spatially across all counties in US
feature_name_to_plot = embedding_features[0]

# Extract the specific feature from the 'features' array column into a new column for plotting
feature_index = int(feature_name_to_plot.replace('feature', ''))
embeddings[feature_name_to_plot] = embeddings['features'].apply(lambda x: x[feature_index])

ax = get_locale(embeddings, county_embeddings.index).plot(feature_name_to_plot)
_ = ax.set_title(feature_name_to_plot + ' in counties')

In [ ]:
county_embeddings.columns

In [ ]:
#@title Map out an embedding dimension feature0 spatially across all counties and zipcodes in NY state
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
state = 'New York'
get_locale(embeddings, county_embeddings.index, states=[state]).plot(feature_name_to_plot, ax=ax[0])
get_locale(embeddings, zip_embeddings.index, states=[state]).plot(feature_name_to_plot, ax=ax[1])
fig.suptitle(f'{feature_name_to_plot} in {state}')
ax[0].set(title='counties')
ax[1].set(title='zip codes')
plt.setp(ax, xticks=[], yticks=[])
fig.tight_layout()

In [ ]:
#@title Map out a prediction variable spatially across all counties and zipcodes in NY state
df = gpd.GeoDataFrame(df, geometry=embeddings.geometry)
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
feature = 'Percent_Person_WithHigherEdu'
state = 'New York'
get_locale(df, county_embeddings.index, states=[state]).plot(feature,
    legend=True, ax=ax[0])
get_locale(df, zip_embeddings.index, states=[state]).plot(feature,
    legend=True, ax=ax[1])
fig.suptitle(f'{feature} in {state}')
ax[0].set(title='counties')
ax[1].set(title='zip codes')
plt.setp(ax, xticks=[], yticks=[])
fig.tight_layout()

# Applying the embeddings in a prediction task

In [ ]:
#@title Common imports and eval methods
import numpy as np
import math
import sklearn.metrics as skmetrics
from sklearn import linear_model, preprocessing
import lightgbm as lgbm

def evaluate(df: pd.DataFrame) -> dict:
    """Evaluates the model performance on the given dataframe.

    Args:
        df: A pandas DataFrame with columns 'y' and 'y_pred'.

    Returns:
        A dictionary of performance metrics.
    """
    # Ensure necessary columns exist and drop rows with NaN or zero in 'y'
    if not {'y', 'y_pred'}.issubset(df.columns):
        raise ValueError("DataFrame must contain 'y' and 'y_pred' columns")

    df = df.dropna(subset=['y', 'y_pred'])
    df = df[df['y'] != 0]

    r2 = skmetrics.r2_score(df['y'], df['y_pred'])
    correlation = df['y'].corr(df['y_pred'])
    rmse = math.sqrt(skmetrics.mean_squared_error(df['y'], df['y_pred']))
    mae = skmetrics.mean_absolute_error(df['y'], df['y_pred'])
    mape = skmetrics.mean_absolute_percentage_error(df['y'], df['y_pred'])

    return {'r2': r2, 'rmse': rmse, 'mae': mae, 'mape': mape, 'correlation': correlation}


def subset_eval(label: str, county_name: str, state: str, gpred: gpd.GeoDataFrame,
                visualize: bool = True, cmap: str = 'Greys') -> dict:
    """Runs intra-county or intra-state evaluation and visualizes the results.

    Args:
        label: The label for the title of the visualization.
        county_name: The specific county name to filter.
        state: The specific state name to filter.
        gpred: GeoDataFrame containing 'y', 'y_pred', 'state', and 'county' columns.
        visualize: Whether to display visualizations.
        cmap: Colormap for visualizations.

    Returns:
        A dictionary of performance metrics.
    """
    # Apply filters based on state and county name
    subset = gpred.copy()
    if state:
        subset = subset[subset['state'] == state]
    if county_name:
        subset = subset[subset['county'] == county_name]

    # Drop rows where 'y' is NaN
    subset = subset.dropna(subset=['y', 'y_pred'])
    eval_metrics = evaluate(subset)

    if visualize:
        _, ax = plt.subplots(1, 3, figsize=(12, 4))

        # Scatter plot of predicted vs actual
        subset.plot.scatter('y', 'y_pred', alpha=0.8, ax=ax[2], color='darkgray')
        x0, x1 = subset[['y', 'y_pred']].min().min(), subset[['y', 'y_pred']].max().max()
        ax[2].plot([x0, x1], [x0, x1], ls='--', color='black')
        ax[2].set_title(f'r={eval_metrics["correlation"]:.2f}, mae={eval_metrics["mae"]:.2f}')

        # Maps of actual and predicted values
        subset.plot('y', legend=True, ax=ax[0], vmin=x0, vmax=x1, cmap=cmap,
                    legend_kwds={'fraction': 0.02, 'pad': 0.05})
        ax[0].set_title('Actual')
        subset.plot('y_pred', legend=False, ax=ax[1], vmin=x0, vmax=x1, cmap=cmap)
        ax[1].set_title('Predicted')

        plt.setp(ax[:2], xticks=[], yticks=[])
        plt.suptitle(f'{label} - {county_name}, {state}')
        plt.tight_layout()

    return eval_metrics


def make_predictions_df(predictions: np.ndarray, test_df: gpd.GeoDataFrame, label: str) -> gpd.GeoDataFrame:
    """Creates a GeoDataFrame with predictions, true labels, and geographic info.

    Args:
        predictions: A sequence of predictions.
        test_df: The original test GeoDataFrame that the predictions are based on.
        label: The column name for the true label in `test_df`.

    Returns:
        A GeoDataFrame for evaluation and visualizations.
    """
    if label not in test_df.columns:
        raise ValueError(f"The specified label '{label}' does not exist in test_df columns.")

    df_predictions = pd.DataFrame({'y': test_df[label], 'y_pred': predictions}, index=test_df.index)
    return test_df[['geometry', 'Count_Person', 'state', 'county']].join(df_predictions)


## Superresolution - Train the model on counties and make predictions for zip code.


In [ ]:
label = 'Percent_Person_WithHigherEdu'
df['population'] = df['location_metadata'].apply(lambda x: x.get('population'))
data = df[df[label].notna() & (df['population'] > 500)]

# Extract individual features from the 'features' array into separate columns in 'data'
# This is crucial for the model to access individual features.
features_df = pd.DataFrame(data['features'].tolist(), index=data.index, columns=embedding_features)
data = data.join(features_df)

train = data[data.index.isin(county_embeddings.index)]
test = data[data.index.isin(zip_embeddings.index)]

model = linear_model.Ridge()
model.fit(train[embedding_features], train[label])
predictions = model.predict(test[embedding_features])
gdf_predictions = make_predictions_df(predictions, test, label)
evaluate(gdf_predictions)

In [ ]:
#@title Visualize some test set predictions
_ = subset_eval(label, 'Harris County', 'Texas', gdf_predictions, cmap='Blues')
_ = subset_eval(label, 'Greenville County', 'South Carolina', gdf_predictions, cmap='Blues')

In [ ]:
#@title Evaluate over a state by setting the county to an empty string.
_ = subset_eval(label, '', 'New York', gdf_predictions, cmap='Blues')

## Imputation - zip -> zip
Train on zipcodes in a subset of counties.

In [ ]:
#@title train on zip codes in 20% of the counties, test on the remaining 80%.
from sklearn.pipeline import make_pipeline
import sklearn.preprocessing

def get_train_test_split(training_fraction=0.8):
  """Splits the data into training and testing sets based on county IDs."""
  data = df[df.index.isin(zip_embeddings.index)].copy()

  # Split the zip codes by county into train/test sets.
  train_counties = data.drop_duplicates('county_id').sample(
      frac=training_fraction).county_id
  train = data[data.county_id.isin(train_counties)].copy()
  test = data[~data.index.isin(train.index)].copy()

  print('# training counties:', len(train_counties),
        '\n# training zip codes:', train.shape[0],
        '\n# test zip codes:', test.shape[0])
  return train, test

def run_imputation_model(
    train,
    test,
    label,
    min_population=500,
    model_class=linear_model.Ridge,
    model_kwargs={}):
  """
  Runs the imputation model.
  """
  train = train[(train.population >= min_population) & train[label].notna()].copy()
  test = test[(test.population >= min_population) & test[label].notna()].copy()

  feature_data_train = train['features'].apply(
      lambda x: pd.Series(x, index=embedding_features))
  feature_data_test = test['features'].apply(
      lambda x: pd.Series(x, index=embedding_features))

  train = pd.concat([train.drop(columns=['features'], errors='ignore'), feature_data_train], axis=1)
  test = pd.concat([test.drop(columns=['features'], errors='ignore'), feature_data_test], axis=1)

  model = make_pipeline(preprocessing.MinMaxScaler(),
                        model_class(**model_kwargs))
  model.fit(train[embedding_features], train[label])
  predictions = model.predict(test[embedding_features])

  gdf_predictions = make_predictions_df(predictions, test, label)
  results = evaluate(gdf_predictions)
  return model, results


# Increasing this value generally improves performance.
training_fraction = 0.2
label = 'Percent_Person_WithHigherEdu'
train, test = get_train_test_split(training_fraction)
model, results = run_imputation_model(train, test, label)
results

In [ ]:
#@title Visualize a few counties from the test set.
test_counties = test.county_id.unique()
large_counties = df[df.county_id.isin(test_counties)].sort_values(
    'population', ascending=False)[['state', 'county', 'population']].head(4)
for _, row in large_counties.iterrows():
  _ = subset_eval(label, row.county, row.state, gdf_predictions, cmap='Blues')

In [ ]:
#@title Try other labels.
labels = [
          'Percent_Person_WithHigherEdu',
          'Median_Age_Person',
          'Median_Income_Household',
          'Percent_Person_WithAsthma',
          'Percent_Person_WithHighBloodPressure'
]
train, test = get_train_test_split(0.8)
models_by_label = {}
metrics_df = pd.DataFrame(
    columns=['label', 'r2', 'rmse', 'mae', 'mape', 'model'])
for label in labels:
  models_by_label[label], results = run_imputation_model(train, test, label)
  results['label'] = label
  results['model'] = 'linear'
  metrics_df.loc[len(metrics_df)] = results

metrics_df.round(3)

In [ ]:
#@title Try LightGBM models instead of linear.

# This will take a few minutes to run.
models_by_label_lgbm = {}
metrics_df_lgbm = pd.DataFrame(
    columns=['label', 'r2', 'rmse', 'mae', 'mape', 'model'])
for label in labels:
  models_by_label_lgbm[label], results = run_imputation_model(
      train, test, label, model_class=lgbm.LGBMRegressor,
      model_kwargs={'min_child_samples': 40,
                    'importance_type': 'gain',
                    'n_estimators': 400,
                    'learning_rate': 0.04,
                    'force_col_wise': True,
                    })
  results['label'] = label
  results['model'] = 'lgbm'
  metrics_df_lgbm.loc[len(metrics_df_lgbm)] = results

metrics_df_lgbm.round(3)

The LGBM results are mostly comparable with the linear model. They can be improved with more iterations and lower learning rate. You can also try setting `feature_fraction=0.5`.

In [ ]:
#@title LightGBM feature importance
import seaborn as sns
features = {
  'trends': (128, embedding_features[:128]),
  'maps': (128, embedding_features[128:256]),
  'weather': (74, embedding_features[256:]),
}
all_importance = []
for label, model in models_by_label_lgbm.items():
  importance = pd.DataFrame(model[1].feature_importances_,
                            index=embedding_features,
                            columns=['importance'])
  importance['importance'] = importance['importance'].abs()
  for feature, dims in features.items():
    importance.loc[dims[1], 'feature'] = feature
  importance = importance.groupby('feature').importance.sum().reset_index()
  importance['importance'] = (importance.importance /
                              importance.importance.sum() * 100)
  importance['label'] = label
  all_importance.append(importance)
all_importance = pd.concat(all_importance)
_, ax = plt.subplots(figsize=(10, 3))
sns.barplot(data=all_importance, x='label', y='importance',
            hue='feature',
            hue_order=features.keys(), ax=ax)
_ = plt.xticks(rotation=30)